In [2]:
!pip install -U sentence-transformers
!pip install transformers

     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 1.3MB 7.8MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
     |████████████████████████████████| 890kB 49.2MB/s 
     |████████████████████████████████| 2.9MB 48.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=cf0ddcba4bdaaff4df7e70d4e4ead3ed5285a94b5c9aaaf408ba5c4d0e54fe52
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=67b0a3b5c0e4a88680515b9bf35a503701ac95c4d72b678b96d18bbad78557cd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import scipy.spatial
import pickle as pkl

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Unzipping the required datafiles move to a folder
!unzip /content/drive/MyDrive/SIG-HAckathon-Submission/data.zip -d /content/drive/MyDrive/SIG-HAckathon-Submission/data/

Archive:  /content/drive/MyDrive/SIG-HAckathon-Submission/data.zip
   creating: /content/drive/MyDrive/SIG-HAckathon-Submission/data/data/
  inflating: /content/drive/MyDrive/SIG-HAckathon-Submission/data/data/corpus_embeddings.pkl  
  inflating: /content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid.csv  
  inflating: /content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid_sentences.csv  
  inflating: /content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid_sentences_Full.csv  


In [4]:
#assign the respective file paths
path_to_covid='/content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid.csv'
path_to_covid_sentences='/content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid_sentences.csv'
path_to_covid_full_sentences='/content/drive/MyDrive/SIG-HAckathon-Submission/data/data/covid_sentences_Full.csv'
path_to_embeddings='/content/drive/MyDrive/SIG-HAckathon-Submission/data/data/corpus_embeddings.pkl'

In [5]:
df_full_sentences=pd.read_csv(path_to_covid_full_sentences)
df_full_sentences.head()

,Unnamed: 0,paper_id,title,abstract,abstract_summary
0,footandmouth disease fmd an acute highly conta...,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
1,the rna virus genome of fmdv displays a very h...,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
2,fmdv virion has a symmetric protein shell or c...,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
3,the fmdv genome was completely sequenced and a...,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
4,the virus capsid consists of 60 copies of each...,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...


In [6]:
df_sentences=pd.read_csv(path_to_covid_sentences)
df_sentences=df_sentences.set_index("Unnamed: 0")
df_sentences.head()

,paper_id
Unnamed: 0,
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 end the orf region is generally divided into four functional areas l p1 p2 and p3 due to the different functions of mature polypeptides orfencoded polyprotein is processed into four products l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro structural proteins form the biological p

In [7]:
df_sentences=df_sentences['paper_id'].to_dict()  #Forming paper_id:Text key value pair
df_sentences_list=list(df_sentences.keys())
len(df_sentences)  #type dictionary

403341

In [8]:
df_sentences_list[-1]

'our study results have the following policy and managerial implications first the mission of healthcare is to promote the healthcare quality our study assures that quality indicator systems facilitate the continuous improvement of healthcare quality second our findings suggest that healthcare quality and efficiency can be achieved simultaneously the advancement in healthcare quality would lead to improvements in healthcare productivity and quality improvement as well third the healthcare quality progress requires transformation of time and input resources table 4 tests of productivity change quality change efficiency change and technical change from pretqip period to posttqip period 19982002 and 19982004 the p values are for testing whether productivity or quality change or relative efficiency change or technical change is significantly less than one significant at the 10 level significant at the 5 level significant at the 1 level for onesided hypothesis tests'

In [9]:
embedder = SentenceTransformer('bert-base-nli-mean-tokens')
corpus = df_sentences_list
#corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
#Using pretrained corpus_embeddings
with open(path_to_embeddings , "rb") as file_:
  corpus_embeddings = pkl.load(file_)

summarizer = pipeline("summarization")

100%|██████████| 405M/405M [00:18<00:00, 22.2MB/s]


In [12]:
def evaluateInput():
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            
            queries=[input_sentence]
            query_embeddings = embedder.encode(queries,show_progress_bar=True)

            ref=df_full_sentences['Unnamed: 0']


            # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
            closest_n = 5
            print("\nTop 5 most similar sentences in corpus:")
            for query, query_embedding in zip(queries, query_embeddings):
                distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

                results = zip(range(len(distances)), distances)
                results = sorted(results, key=lambda x: x[1])
                text=" "
                for idx,dist in results[:5]:
                  print("Paragraph: ", corpus[idx],"\n")
                  indexes=[]
                  para=corpus[idx].replace(' ','')
                  for i in range(len(ref)):
                    if str(ref[i]).replace(' ','')==str(para):
                      indexes.append(i)

                  for id in indexes:
                    row_dict = df_full_sentences.loc[df_full_sentences.index==id].to_dict()
                    #print("paper_id:  " , row_dict["paper_id"] , "\n")
                    #print("Title:  " , row_dict["title"] , "\n")
                    #print("Abstract:  " , row_dict["abstract"] , "\n")
                    #print("Abstract_Summary:  " , row_dict["abstract_summary"] , "\n")
                    text+=str(row_dict["abstract_summary"])
                    #print(summarizer(row_dict[], max_length=150, min_length=30))
                    
                    indexes=[]   

            summarized_text=summarizer(text, max_length=250, min_length=30)

            return summarized_text

        except KeyError:

            print("Error Occured!!")

In [13]:
def wrap(s, n):

  a = s.split()
  ret = ''
  for i in range(0, len(a), n):
    ret += ' '.join(a[i:i+n]) + '\n'
  return ret
  
while(1):
  res=evaluateInput()
  if res is None:
    break
  else:
    output=res[0]['summary_text']
    output=wrap(output,10)
    print("Bot:", output)


> What are the ways to reduce virus infections




Top 5 most similar sentences in corpus:
Paragraph:  to reduce the incidence of vaccine preventable diseases 

Paragraph:  as well as stockpiling antivirals it may be possible to reduce transmission and severity of disease by stockpiling a partiallybox 1 epidemicspecific characteristics affecting suitability of interventions 

Paragraph:  to reduce transmission of infectious diseases 

Paragraph:   provide public health guidance to reduce opportunistic infections among immunocompromised persons 

Paragraph:  in order to prevent exacerbations the design of new therapeutics could potentially improve efficacy by both directly acting to inhibit viral replication and alleviate the symptoms of asthma and copd [51]  

Bot: All aspects of the heath care sector are being asked
to account for their performance . This poses unique challenges
for local public health units with their traditional focus on
disease prevention . In 2004 the provincial Institute for Clinical
<br>Evaluative Sciences bas



Top 5 most similar sentences in corpus:
Paragraph:  as well as stockpiling antivirals it may be possible to reduce transmission and severity of disease by stockpiling a partiallybox 1 epidemicspecific characteristics affecting suitability of interventions 

Paragraph:  interventions to reduce transmission can include community mobilization quarantine isolation treatment or vaccination the potential success of these interventions is determined by general characteristics of the disease such as overall transmissibility and how this varies across populations [43]  furthermore certain types of interventions may be more or less appropriate depending on the natural history of the disease and the context of the epidemic 

Paragraph:  to reduce the incidence of vaccine preventable diseases 

Paragraph:   goals to be achieved for endemic and epidemic infections  development of recommendations for ipc processes and practices that are known to be effective in preventing hai and the spread of amr

KeyboardInterrupt: ignored

In [ ]:
"""
Observations:

1. Experimentations were done on using CORD-19 dataset topics (TREC-COVID), and feeding the GPT2 Model the form of Question and Answers(relevant docs). However the generated text failed to convince as acceptable answer
2. Using the BERT embedding seemed to give better results over here, however CORD-19 being such a huge dataset itself, creating the embeddings for ~134000 json files abstract text was a very heavy computational task on local systems.
   The embeddings can be trained better and fine tuned more suitably experiment with different preprocessing techniques and text representations, if computation wasn't a roadbloack for us.
3. The bot results above doesn't seem to be always contextual, as the articles represent more of a sophistacted language compared to general human terms. Hence it answers to queries in more a medical scholar style!!

"""

"\nObservations:\n\n1. Experimentations were done on using CORD-19 dataset topics (TREC-COVID), and feeding the GPT2 Model the form of Question and Answers(relevant docs). However the generated text failed to convince as acceptable answer\n2. Using the BERT embedding seemed to give better results over here, however CORD-19 being such a huge dataset itself, creating the embeddings for ~134000 json files abstract text was a very heavy computational task on local systems.\n   The embeddings can be trained better and fine tuned more suitably experiment with different preprocessing techniques and text representations, if computation wasn't a roadbloack for us.\n3. The bot results above doesn't seem to be always contextual, as the articles represent more of a sophistacted language compared to general human terms. Hence it answers to queries in more a medical scholar style!!\n\n"